## Pre-processing

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

In [3]:
print("The maximum length of review : ", max(len(l) for l in X_train))
print("The mean length of review : ",sum(map(len, X_train))/len(X_train))

The maximum length of review :  2494
The mean length of review :  238.71364


In [4]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

## Model Building

In [5]:
import tensorflow as tf

In [6]:
class BahdanauAttention(tf.keras.Model) :
    def __init__(self, units) :
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
    
    def call(self, values, query) :
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

In [7]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [9]:
sequence_input = Input(shape=(max_len,), dtype="int32")
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

In [10]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

In [12]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [13]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [14]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

In [18]:
attention = BahdanauAttention(64)
context_vector, attention_weights = attention(lstm, state_h)

In [19]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [20]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
history = model.fit(X_train, y_train, epochs = 4, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/4
98/98 [==============================] - 624s 6s/step - loss: 0.4644 - accuracy: 0.7714 - val_loss: 0.3104 - val_accuracy: 0.8738
Epoch 2/4
98/98 [==============================] - 643s 7s/step - loss: 0.2223 - accuracy: 0.9210 - val_loss: 0.2923 - val_accuracy: 0.8796
Epoch 3/4
98/98 [==============================] - 647s 7s/step - loss: 0.1414 - accuracy: 0.9541 - val_loss: 0.3596 - val_accuracy: 0.8724
Epoch 4/4
98/98 [==============================] - 643s 7s/step - loss: 0.1052 - accuracy: 0.9681 - val_loss: 0.4176 - val_accuracy: 0.8685


In [23]:
print("Test accuracy: %.4f" %(model.evaluate(X_test,y_test)[1]))

782/782 [==============================] - 663s 847ms/step - loss: 0.4176 - accuracy: 0.8685
Test accuracy: 0.8685
